In [1]:
!pip install -r requirement.txt

In [2]:
import numpy as np
import geatpy as ea
import pandas as pd
import random
from datetime import datetime
print(ea.__version__)

2.7.0


### READ DATA
TODO: read data from GP

In [3]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')

In [4]:
# randomly choose 1000 samples
data = rawData.iloc[:30000, :61].sample(1000)
print(data.shape)
data.sample(3)

(1000, 61)


,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,...,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units
12575,FZOPS,DF3RM,FFAU3691819,4420.0,DFIC-FL-2104,3/3/2021,1690478.0,DFC3,D4H,FFAU3691819,...,1.7,2021 NP,Reasonable,3.520164,10.423562,from ANZ portfolio,LE-ONE-09BD4H,NaN,1499.0,1.0
28857,FZOPS,DF3RM,FBIU0105674,2182.0,DFIC-FL-1801,1/2/2018,1080751.0,DFC3,D20,FBIU0105674,...,1.0,2018 NP,Reasonable,5.240685,6.759315,from available portfolio,LF-HAPAG-09D20,NaN,1494.0,1.0
11812,FZOPS,DF3RM,FFAU3699790,4420.0,DFIC-FL-2104,3/3/2021,1691276.0,DFC3,D4H,FFAU3699790,...,1.7,2021 NP,Reasonable,3.520164,10.423562,from ANZ portfolio,LE-ONE-09BD4H,NaN,1499.0,1.0


### DATA PREPARATION

In [5]:
# TODO: add more columns later
input_data = \
    data.loc[:, ['Cost', 'Mfr Dt Fz', 'Product', 'Contract Cust Id', 'Contract Dt', 'Contract Expiration Dt', 'Nbv', 'Billing Status Fz']]
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
23886,2267.0,3/22/2018,D20,ONE,2018-02-01,2026-10-14,1896.167,ON
23377,2267.0,3/22/2018,D20,ONE,2018-02-01,2026-10-14,1896.088,ON
22530,2267.0,3/22/2018,D20,ONE,2018-02-01,2026-10-14,1888.004,ON


In [6]:
# select standard equipment
standardEquipment = ['D4H', 'D20', 'D40']
def SelectStandardEquip(Product):
    if Product in standardEquipment:
        Product = 1
    else:
        Product = 0
    return Product

input_data['Product'] = input_data.apply(lambda x: SelectStandardEquip(x['Product']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
22308,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1888.001,ON
6298,3682.0,1/1/2018,1,MSC,2018-03-01,2032-07-31,2933.000,ON
27750,3682.0,1/3/2018,1,HAPAG,2017-11-01,2030-01-31,2869.540,ON


Find Top 1 and Top 3 Lessees

In [7]:
# select the Top1 lessee
input_data.groupby('Contract Cust Id').count().sort_values('Cost', ascending=False)

,Cost,Mfr Dt Fz,Product,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
Contract Cust Id,,,,,,,
MSC,406,406,406,406,406,406,406
ONE,359,359,359,359,359,359,359
HAPAG,211,211,211,211,211,211,211
COSMR,7,7,7,7,7,7,7
TCLC,7,7,7,7,7,7,7
CMA,5,5,5,5,5,5,5
PANOCEAN,5,5,5,5,5,5,5


In [8]:
# top1 and top3 lessees 
Top1Lessee = 'MSC'
Top3Lessees = ['MSC', 'ONE', 'HAPAG']

#### Select top 1 and top 3 lessees
Assign new column `Top1Lessee` and `Top3Lessees`

In [9]:
def SelectTop1(custID):
    if custID == Top1Lessee:
        custID = 1
    else:
        custID = 0
    return custID

def SelectTop3(custID):
    if custID in Top3Lessees:
        custID = 1
    else:
        custID = 0
    return custID

input_data['Top1Lessee'] = input_data.apply(lambda x: SelectTop1(x['Contract Cust Id']), axis=1)
input_data['Top3Lessees'] = input_data.apply(lambda x: SelectTop3(x['Contract Cust Id']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees
13329,6007.0,4/28/2021,1,MSC,2021-04-01,2029-06-30,5618.80,ON,1,1
5447,3682.0,2/28/2018,1,MSC,2018-03-01,2032-07-31,2933.00,ON,1,1
7409,3702.0,6/11/2018,1,MSC,2018-03-01,2032-07-31,2983.22,ON,1,1


#### Calcualte manufacture age
Assign new column `MfrAge`

In [10]:
# TODO: consider timezone, consider time_now in the future
time_now = datetime.now()
# calculate manufacture age
date_format = '%m/%d/%Y'
# return in years, round to 2 decimal
def CalculateManufacutreAge(date):
    diff = time_now.date() - datetime.strptime(date, date_format).date()
    return round(diff.days / 365, 2)

input_data['MfrAge'] = input_data.apply(lambda x: CalculateManufacutreAge(x['Mfr Dt Fz']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge
7851,3702.0,6/11/2018,1,MSC,2018-03-01,2032-07-31,2983.214,ON,1,1,4.29
2293,3832.0,12/18/2017,1,HAPAG,2017-11-01,2030-01-31,2981.794,ON,0,1,4.77
11246,4420.0,3/24/2021,1,ONE,2020-11-16,2029-05-15,4155.250,ON,0,1,1.50


#### Calculate unexpired year 
Assign new column `ExpYear`

In [13]:
# TODO: consider timezone, consider time_now in the future
# TODO: 'Contract Expiration Dt' is in Timestamp, the data type may change
time_now = datetime.now()
# calculate manufacture age
date_format = '%Y-%m-%d'
# return in years, round to 2 decimal
def CalculateExpYear(date):
    diff = pd.to_datetime(date,format="%Y%m%d").date() - time_now.date()
    return round(diff.days / 365, 2)

input_data['ExpYear'] = input_data.apply(lambda x: CalculateExpYear(x['Contract Expiration Dt']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
21896,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1888.004,ON,0,1,4.51,4.06
28683,2182.0,1/2/2018,1,HAPAG,2017-11-01,2030-01-31,1784.484,ON,0,1,4.73,7.36
29913,2182.0,1/2/2018,1,HAPAG,2017-11-01,2030-01-31,1784.490,ON,0,1,4.73,7.36


Save data

In [14]:
input_data.to_csv('prepared_data.csv')

In [15]:
prepared_data = pd.read_csv('./prepared_data.csv')
prepared_data.head(3)

,Unnamed: 0,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
0,20218,3849.0,4/6/2018,1,ONE,2018-02-01,2026-10-14,3123.450,ON,0,1,4.47,4.06
1,18248,6000.0,4/12/2021,1,MSC,2021-04-01,2029-06-30,5612.350,ON,1,1,1.45,6.77
2,7790,3702.0,6/11/2018,1,MSC,2018-03-01,2032-07-31,2983.211,ON,1,1,4.29,9.86


### Model Part

#### Input

In [ ]:
# TODO: set input


#### Define Problem
Define objective function and constraint functions

In [ ]:
class PortfolioSelection(ea.Problem):
    def __init__(self):
        name ='PortfolioSelection' # name o problem
        M = 1 # variable dimension; set to 1
        maxormins = [-1] # 1--min, -1--max
        Dim = 1000 # num of variables--num of containers
        varTypes = [1] * Dim # binary: 0 or 1
        lb = [0,0,0] # low bound
        ub = [1,1,1] # up bound
        lbin = [1,1,1] # low bound include
        ubin = [1,1,1] # up bound include
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb,ub, lbin, ubin)
        
    # objective and constraints
    def aimFunc(self, pop):
        Vars = pop.Phen # (NIND, Dim)
        
        # objective function
        pop.ObjV = 4*x1 + 2*x2 + x3
        # constraints
        pop.CV = np.hstack([
            2*x1 + x2 - 1,
            x1 + 2*x3 - 2,
            np.abs(x1 + x2 + x3 - 1)])


Set Parameters

In [ ]:
problem = PortfolioSelection()
# init population
Encoding ='RI' # TODO: may be change later
NIND = 50 # init num of population
Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges,problem.borders)
population = ea.Population(Encoding, Field, NIND)
# init algorithm
myAlgorithm = ea.soea_DE_best_1_L_templet(problem, population)
myAlgorithm.MAXGEN = 1000 # max iteration 
myAlgorithm.mutOper.F = 0.5 # TODO: may be change later
myAlgorithm.recOper.XOVR = 0.7 # TODO: may be change later
myAlgorithm.logTras = 1 # write log
myAlgorithm.verbose = True
myAlgorithm.drawing = 1 # 1--show result, 2--show process
# TODO: consider 'trappedValue', 'maxTrappedCount' and 'seed'



Run

In [ ]:
[BestIndi, population] = myAlgorithm.run()
# BestIndi.save()
print('Evaluation number：%s'% myAlgorithm.evalsNum)
print('Time cost: %s seconds'% myAlgorithm.passTime)
if BestIndi.sizes != 0:
    print('Optimal objective function value: %s'% BestIndi.ObjV[0][0])
    print('Optimal variable value: ')
    for i in range(BestIndi.Phen.shape[1]):
        print(BestIndi.Phen[0, i])
else:
    print('No feasible results.')